In [ ]:
# 1. Import knihoven
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# 2. Načti katalog
rohlik = pd.read_csv('data/rohlik.csv')

In [ ]:
# 3. Načtení ingrediencí a metadat ze souboru
ingredience_df = pd.read_csv("data/ingredience.csv")
ingredience_df = ingredience_df.dropna(subset=["ingredience_nazev"]).copy()
ingredience_df = ingredience_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

ingredience = ingredience_df["ingredience_nazev"].tolist()

mapovani_ingredienci_na_kategorie = {
    row["ingredience_nazev"]: str(row["klicova_slova_kategorie"]).split(", ")
    for _, row in ingredience_df.iterrows()
    if pd.notna(row["klicova_slova_kategorie"])
}

blacklist_nazvu = {
    row["ingredience_nazev"]: str(row["blacklist_slov_nazev"]).split(", ")
    for _, row in ingredience_df.iterrows()
    if pd.notna(row["blacklist_slov_nazev"])
}

blacklist_kategorii = {
    row["ingredience_nazev"]: str(row["blacklist_slov_kategorie"]).split(", ")
    for _, row in ingredience_df.iterrows()
    if pd.notna(row["blacklist_slov_kategorie"])
}


/var/folders/g3/_j9jhm8x5nzd__2b6r8pvkq40000gn/T/ipykernel_58556/3855198147.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ingredience_df = ingredience_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# 4. Načtení embedding modelu
model = SentenceTransformer('all-MiniLM-L6-v2')

In [40]:
# 5. Vytvoření embeddingů pro ingredience
ingredience_embeddingy = model.encode(ingredience, show_progress_bar=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 6. FUnkce - Kontrola kategorie: respektuje klíčová slova i blacklist
def kategorie_souhlasi(row, klicova_slova, blacklist_kat):
    # 1. Původní hodnoty pro kontrolu prázdnosti
    raw_kategorie = [row.get(cat, None) for cat in ['cat4', 'cat3', 'cat2', 'cat1']]
    
    # 2. Pokud všechny kategorie jsou prázdné nebo NaN → označ jako 'no_category'
    if all(pd.isna(k) or k == '' for k in raw_kategorie):
        return 'no_category'

    # 3. Sloučené stringy pro kontrolu obsahu
    kategorie_texty = [str(k).lower() for k in raw_kategorie]
    spojene_kategorie = " ".join(kategorie_texty)

    if any(black in spojene_kategorie for black in blacklist_kat):
        return False
    if any(slovo in spojene_kategorie for slovo in klicova_slova):
        return True
    return False

# 7. Funkce: zakázaná slova v názvu
def je_povoleny_nazev(nazev, blacklist):
    nazev = str(nazev).lower()
    return not any(slovo in nazev for slovo in blacklist)

In [ ]:
# 8. Vyhledání nejlepšího produktu pro každou ingredienci
vysledky = []
for idx, ingredience_nazev in enumerate(ingredience):
    klicova_slova = mapovani_ingredienci_na_kategorie.get(ingredience_nazev, [])

    zakazana_slova = blacklist_nazvu.get(ingredience_nazev, [])
    zakazane_kategorie = blacklist_kategorii.get(ingredience_nazev, [])

    
    # Vytvoření embeddingu pro ingredienci
    ingredience_emb = ingredience_embeddingy[idx]

    # Vyhodnocení kategorií
    rohlik['kategorie_match'] = rohlik.apply(lambda row: kategorie_souhlasi(row, klicova_slova, zakazane_kategorie), axis=1)

    rohlik['kategorie_match'] = rohlik['kategorie_match'].apply(lambda x: x if x == 'no_category' else bool(x))

    
    # Vybereme kandidáty (odpovídající kategorie nebo prázdná kategorie)
    kandidati = rohlik[(rohlik['kategorie_match'] == True) | (rohlik['kategorie_match'] == 'no_category')].copy()

    # Filtrování podle zakázaných slov v názvu
    kandidati = kandidati[kandidati['itemName'].apply(lambda x: je_povoleny_nazev(x, zakazana_slova))]

    if kandidati.empty:
        print(f"Žádný vhodný produkt pro {ingredience_nazev}")
        continue

    # Vytvoření embeddingů názvů kandidátů
    kandidati_nazvy = kandidati['itemName'].tolist()
    kandidati_embeddingy = model.encode(kandidati_nazvy, show_progress_bar=False)
    
    # Spočítání podobností
    podobnosti = cosine_similarity([ingredience_emb], kandidati_embeddingy)[0]
    kandidati['podobnost'] = podobnosti

    # Rozlišení prahu podobnosti:
    def vhodny_prah(row):
        if row['kategorie_match'] == 'no_category':
            return 0.65  # bez kategorie chceme vyšší podobnost
        else:
            return 0.4

    kandidati['prah_podobnosti'] = kandidati.apply(vhodny_prah, axis=1)
    kandidati = kandidati[kandidati['podobnost'] >= kandidati['prah_podobnosti']]
    
    if kandidati.empty:
        print(f"Žádný dost podobný produkt pro {ingredience_nazev}")
        continue

    # Seřadíme podle nejvyšší podobnosti
    kandidati = kandidati.sort_values(by=['podobnost'], ascending=[False])

    # Vybereme 15 nejlepších kandidátů
    top_kandidati = kandidati.head(20)

    for _, produkt in top_kandidati.iterrows():
        vysledky.append({
            "Ingredience": ingredience_nazev,
            "Produkt": produkt['itemName'],
            "ID": produkt['id'],
            "URL": produkt['itemUrl'],
            "IMG": produkt['img'],
            "Cena": produkt['price'],
            "Jednotková cena": produkt['unit_price'],
            "Jednotka": produkt['unit'],
            "Velikost balení": produkt['quantity_value'],
            "Jednotka balení": produkt['quantity_unit'],
            "Kategorie": produkt['category'],
            "Podobnost": round(produkt['podobnost'], 3),
            "Původ kategorie": produkt['kategorie_match']
        })

In [ ]:
# 9. Výstup výsledků
vysledky_df = pd.DataFrame(vysledky)

# 10. Uložení výsledků
vysledky_df.to_csv('output/p_01_ingredience_rohlik_predfiltrovane.csv', index=False)